### Import Lib

In [1]:
import os
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from data_preprocessing import NusaXSentimentDataProcessor
from LSTM.lstm import LSTMModel
from RNN.rnn import RNNModel



### Import Dataset

In [2]:
data_dir = '../indonesian'
data_processor = NusaXSentimentDataProcessor(data_dir,sequence_length = 30)# 30 timesteps
print("Preparing data...")
(x_train, y_train), (x_val, y_val), (x_test, y_test) = data_processor.prepare_data()



Preparing data...
Unique labels found: {'negative', 'positive', 'neutral'}
Train data: 500 samples
Validation data: 100 samples
Test data: 400 samples


### Making Keras Model Architecture 

In [3]:
from tensorflow import keras
import tensorflow as tf

def create_and_train_model(x_train, 
                           y_train, 
                           x_val, 
                           y_val, 
                           vocab_size, 
                           num_classes, 
                           model_type='lstm',
                           embedding_dim = 100, 
                           hidden_units=25,
                           epoch=25,
                           batch_size=32):
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, embedding_dim))

    # Pilih jenis RNN
    if model_type.lower() == 'lstm':
        model.add(keras.layers.LSTM(hidden_units))
    elif model_type.lower() == 'simplernn':
        model.add(keras.layers.SimpleRNN(hidden_units))
    else:
        raise ValueError("model_type harus salah satu dari: 'lstm', 'simplernn")

    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    print(f"\nTraining {model_type.upper()} model with {num_classes} classes...")
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=epoch,
        batch_size=batch_size
    )

    return model, history

def preprocess_text_for_prediction(processor: NusaXSentimentDataProcessor, text: str):
    if processor.vectorize_layer is None:
        raise ValueError("Vectorize layer belum diadaptasi. Jalankan prepare_data() dulu.")

    vectorized = processor.vectorize_text(tf.constant([text]))
    return vectorized.numpy()

In [4]:
vocab_size = data_processor.get_vocabulary_size()
num_classes = data_processor.get_num_classes()
print(f"Vocab Size: {vocab_size} Num Classes {num_classes}")
print(vocab_size)

Vocab Size: 2836 Num Classes 3
2836


#### LSTM

In [5]:
model_lstm,hist = create_and_train_model(x_train,y_train,x_val,y_val,vocab_size,num_classes,'lstm',epoch=25)
model_lstm.summary()
#rumus param lstm: (1+10+1)*4*4 = 480
#rumus dense output: (10+1) * 3 = 33



Training LSTM model with 3 classes...
Epoch 1/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.4372 - loss: 1.0847 - val_accuracy: 0.5000 - val_loss: 1.0507
Epoch 2/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4803 - loss: 1.0120 - val_accuracy: 0.5500 - val_loss: 0.9433
Epoch 3/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5993 - loss: 0.8500 - val_accuracy: 0.5400 - val_loss: 0.8734
Epoch 4/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7269 - loss: 0.7102 - val_accuracy: 0.6000 - val_loss: 0.8118
Epoch 5/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8528 - loss: 0.5228 - val_accuracy: 0.6300 - val_loss: 0.8945
Epoch 6/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9191 - loss: 0.3265 - val_accuracy: 0.6400 - val_loss: 1.0423
Epoch 7/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9640 - loss: 0.1904 - val_accuracy: 0.6600 - val_loss: 1.0965
Epoch 8/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9614 - los

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 30, 100)        │       283,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 25)             │        12,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            78 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 888,836 (3.39 MB)

 Trainable params: 296,278 (1.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 592,558 (2.26 MB)

In [6]:
custom_model_lstm = LSTMModel(model_lstm) 
custom_model_lstm.print_info()


Model Architecture Information:

Layer 0: Embedding
------------------------
E (Embedding Matrix):
  Shape: (2836, 100)
  - rows: vocabulary size (|V|)
  - cols: embedding dimension (d)

Config:
{'name': 'embedding', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 2836, 'output_dim': 100, 'embeddings_initializer': {'module': 'keras.initializers', 'class_name': 'RandomUniform', 'config': {'seed': None, 'minval': -0.05, 'maxval': 0.05}, 'registered_name': None}, 'embeddings_regularizer': None, 'activity_regularizer': None, 'embeddings_constraint': None, 'mask_zero': False}

Layer 1: LSTM
------------------------
Weight Matrices:
W (Input Weight Matrix):
  Shape: (100, 100)
  - rows: input dimension (d)
  - cols: 4*h where h is hidden size (for i,f,g,o gates)

U (Recurrent Weight Matrix):
  Shape: (25, 100)
  - rows: hidden size (h)
  - cols: 4*h (for i,f,g,o gates)

b (Bias Vector):
  Shap

In [7]:
y_keras = model_lstm.predict(x_test)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [8]:
y_scratch = custom_model_lstm.forward(x_test)

total timestep: 30


In [9]:
f1_scratch, y_pred_scratch, outs = custom_model_lstm.evaluate(x_test,y_test)

total timestep: 30
F1 Score (macro): 0.6036


In [10]:
print(y_keras[0:1])
print(y_scratch[0:1])
print(y_test)

[[0.32831162 0.00472465 0.66696376]]
[[0.98213553 0.00428507 0.01357941]]
[2 1 0 2 1 0 1 0 2 2 1 0 0 1 0 0 1 0 1 2 1 0 2 1 1 2 2 0 2 2 1 1 0 2 0 0 0
 0 2 2 2 0 0 0 1 1 1 1 1 2 0 2 0 2 1 0 2 1 0 2 1 2 1 1 0 1 2 0 0 0 1 0 1 1
 2 0 2 2 2 2 0 0 1 2 2 2 0 2 0 0 0 2 1 1 2 0 1 2 0 1 0 2 1 2 0 1 0 2 2 2 1
 2 2 2 0 1 2 2 2 2 2 0 2 1 0 0 2 0 0 1 1 1 1 0 2 0 2 2 2 2 2 0 1 0 2 2 1 2
 0 1 1 2 1 2 1 1 0 1 2 2 2 0 2 0 2 2 2 1 0 1 2 0 2 0 2 1 0 2 1 2 0 2 1 0 0
 0 1 0 1 2 1 2 0 0 1 2 2 0 0 0 2 0 1 2 2 1 2 0 0 0 2 0 2 0 2 1 0 0 2 1 0 1
 0 2 2 0 1 2 1 2 0 1 1 0 1 0 0 0 2 1 0 2 0 0 2 1 0 1 0 2 1 2 0 2 2 1 1 1 2
 2 0 2 2 0 1 0 2 2 2 0 0 0 2 2 2 0 2 0 2 2 2 2 2 2 0 2 0 2 1 0 2 0 0 1 2 1
 0 0 0 0 1 2 1 0 2 2 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 2 0 2 0 2 0 1 2 0 2 2 2
 1 1 2 0 0 0 0 2 0 0 0 0 0 2 0 0 0 2 1 0 2 1 1 0 2 0 2 0 1 2 2 0 0 2 2 2 1
 0 2 2 2 0 2 2 2 2 0 1 0 0 1 2 1 1 0 2 0 0 0 2 2 0 2 0 2 0 0]


In [11]:
from sklearn.metrics import f1_score
y_keras_classes = np.argmax(y_keras, axis=1)
f1_keras = f1_score(y_test, y_keras_classes, average='macro')  
print("F1 Score Keras:", f1_keras)
print("F1 Score Scratch: ", f1_scratch)


F1 Score Keras: 0.674402172590174
F1 Score Scratch:  0.6036195286195286


In [12]:
input_text = "zaki sangat suka makan sayur"

input_vector = preprocess_text_for_prediction(data_processor, input_text)
y_pred_probs = model_lstm.predict(input_vector)
predicted_label = y_pred_probs.argmax(axis=1)[0]
reverse_label_mapping = {v: k for k, v in data_processor.label_mapping.items()}
predicted_class_name = reverse_label_mapping[predicted_label]

print(f"Prediksi kalimat '{input_text}' adalah: {predicted_class_name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediksi kalimat 'zaki sangat suka makan sayur' adalah: negative


#### SimpleRNN

In [13]:

model_rnn,hist = create_and_train_model(x_train,y_train,x_val,y_val,vocab_size,num_classes,'simplernn')
model_rnn.summary()



Training SIMPLERNN model with 3 classes...
Epoch 1/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4280 - loss: 1.0686 - val_accuracy: 0.5300 - val_loss: 0.9901
Epoch 2/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6695 - loss: 0.8210 - val_accuracy: 0.5100 - val_loss: 0.9456
Epoch 3/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9420 - loss: 0.5397 - val_accuracy: 0.5900 - val_loss: 0.8975
Epoch 4/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9916 - loss: 0.3047 - val_accuracy: 0.6300 - val_loss: 0.9084
Epoch 5/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9964 - loss: 0.1551 - val_accuracy: 0.5800 - val_loss: 0.9750
Epoch 6/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.1064 - val_accuracy: 0.5900 - val_loss: 0.9982
Epoch 7/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0665 - val_accuracy: 0.5900 - val_loss: 1.0144
Epoch 8/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - lo

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 30, 100)        │       283,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 25)             │         3,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 25)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            78 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 860,486 (3.28 MB)

 Trainable params: 286,828 (1.09 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 573,658 (2.19 MB)

In [14]:
custom_model_rnn = RNNModel(model_rnn) 
custom_model_rnn.print_info()


Model Architecture Information:

Layer 0: Embedding
------------------------
E (Embedding Matrix):
  Shape: (2836, 100)
  - rows: vocabulary size (|V|)
  - cols: embedding dimension (d)

Config:
{'name': 'embedding_2', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 2836, 'output_dim': 100, 'embeddings_initializer': {'module': 'keras.initializers', 'class_name': 'RandomUniform', 'config': {'seed': None, 'minval': -0.05, 'maxval': 0.05}, 'registered_name': None}, 'embeddings_regularizer': None, 'activity_regularizer': None, 'embeddings_constraint': None, 'mask_zero': False}

Layer 1: SimpleRNN
------------------------
Weight Matrices:
W (Input Weight Matrix):
  Shape: (100, 25)
  - rows: input dimension (d)
  - cols: hidden size (h)

U (Recurrent Weight Matrix):
  Shape: (25, 25)
  - rows: hidden size (h)
  - cols: hidden size (h)

b (Bias Vector):
  Shape: (25,)
  - size: hidden size (h)

In [15]:
y_keras = model_rnn.predict(x_test[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


In [16]:
#tinggal predict disini
# y_scratch = custom_model_rnn.predict(x_test[0:1])